# SDTM mapping assistant

## Introduction

This notebook demonstrates an end-to-end pipeline to automate Study Data Tabulation Model (SDTM) for mapping used for reporting of clinical studies results to the FDA in the US. This is a challenging task for biostatistians as it's effort-intensive, cognitively demanding, and requires full verification of accuracy and completeness of the SDTM dataset for regulatory compliance. In many cases this activity leads to fatigue and burnout. 

### What we'll cover

1. Environment setup
2. Study data generator - Since we're looking at CDMS as input, we'll ask GPT to generate CDMS synthetic dataset. 
3. Study explorer - Discover missing domains, missing fields, or formatting issues upfront.
4. SDTM mapping agent - Dynamic automapper agent creating a draft mapping specification with trace/chain of thought record. 
In this case, we configure the agent for SDTMIG 3.3.
5. SDTM validator agent - automatically verifies and updates the mappings. As with the previous steps, lineage/traceability of record changes is critical and should not be broken.

## 1. Environment setup
1.1 Install the necessary libraries
1.2 Import the necessary libraries
1.3 Set up API keys

In [1]:
%pip install pandas numpy scikit-learn torch transformers matplotlib openai
%pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 1.3 MB/s eta 0:00:00-:--:--
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0

[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import json
import pandas as pd
import os
import re

In [3]:
#########
# First, create a `.env` file in the same directory as this file with the following content:
# OPENAI_API_KEY=your_api_key_here
# Then install the python-dotenv if you haven't
# %pip install python-dotenv
# Then run the following code block
#########

from dotenv import load_dotenv
from openai import OpenAI

# Specify path to the env file, in our case: `cred.env` file in the current directory
load_dotenv("cred.env",override=True)

# Retrieve the API key and set it to openai_api_key variable
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client with the loaded key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Verify that the API key is loaded successfully
print("API key loaded:", bool(client.api_key))

API key loaded: True


In [4]:
# Check available openAI models 
models = client.models.list()
for m in models.data:
    print(m.id)

gpt-4o-audio-preview-2024-12-17
dall-e-3
dall-e-2
gpt-4o-audio-preview-2024-10-01
text-embedding-3-small
o4-mini
gpt-4.1-nano
gpt-4.1-nano-2025-04-14
gpt-4o-realtime-preview-2024-10-01
o4-mini-2025-04-16
gpt-4o-realtime-preview
babbage-002
gpt-4
text-embedding-ada-002
chatgpt-4o-latest
text-embedding-3-large
gpt-4o-mini-audio-preview
gpt-4o-audio-preview
o1-preview-2024-09-12
gpt-4o-mini-realtime-preview
gpt-4.1-mini
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct-0914
gpt-4o-mini-search-preview
gpt-4.1-mini-2025-04-14
davinci-002
gpt-3.5-turbo-1106
gpt-4o-search-preview
gpt-4-turbo
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct
gpt-3.5-turbo
gpt-4-turbo-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4-0125-preview
gpt-4o-2024-11-20
whisper-1
gpt-4o-2024-05-13
gpt-4-turbo-2024-04-09
gpt-3.5-turbo-16k
gpt-image-1
o1-preview
gpt-4-0613
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-search-preview-2025-03-11
omni-moderation-2024-09-26
o3-mini
o3-mini-2025-0

## 2. Study data generator

In this section we load the clinical data. Since we're looking at CDMS as input, we'll ask GPT to generate CDMS synthetic dataset. 

### Generate dataset

In [5]:
prompt_version_cdms_dataset = "v1"  # version of the prompt design. bump with each update

prompt_cdms_dataset = """
You are a clinical data manager.

Generate synthetic, realistic raw CDMS datasets for a clinical trial covering the following domains:
- DM (Demographics)
- VS (Vital Signs)
- AE (Adverse Events)
- EX (Exposure)
- TS (Trial Summary)
- TV (Trial Visits)

Requirements:
- Create realistic and complete sets of fields typically collected at the CDMS level based on your domain knowledge.
- Do NOT limit the number of fields; reflect actual clinical trialdata collection practices and include diverse metadata (e.g., demographics, vital measurements, event severities, dosing routes).
- Output each domain as a realistic **CSV format** block: a header row followed by data rows.
- Each domain should include 10-20 records, multiple subjects, and multiple visits where appropriate.
- Include missing/null values in some fields realistically.
- Clearly label each section with "DM Dataset:", "VS Dataset:", "AE Dataset:", etc.

Respond only with the CSV content clearly separated by domain - no explanations, introductions, or notes.

Be as accurate and complete as possible.
"""

In [6]:
# Initialize client (reads API key from env var or config)
client = OpenAI()
model_name = "o4-mini"

In [7]:
#Function wrapper to callGPT and get CDMS dataset,for a given prompt `prompt_cdms_dataset`
def generate_cdms_dataset(prompt_cdms_dataset):
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt_cdms_dataset}],
            # temperature=0.1, # option not available with latest reasoning models
            # max_tokens=4000  # option not available with latest reasoning models
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating explanation: {e}"

In [21]:
# Generate GPT response with CDMS dataset
cdms_csv_text = generate_cdms_dataset(prompt_cdms_dataset)
print("Generated CDMS dataset.")
print(cdms_csv_text)

Generated CDMS dataset.
DM Dataset:
STUDYID,DOMAIN,USUBJID,SUBJID,SITEID,RFSTDTC,RFENDTC,DMDTC,BRTHDTC,AGE,AGEU,SEX,RACE,ETHNIC,ARM,ARMCD,COUNTRY
STUDY001,DM,STUDY001-101,101,SITE01,2022-01-10,2022-06-10,2022-01-05,1980-04-12,41,Y,M,White,Not Hispanic or Latino,Treatment A,A1,USA
STUDY001,DM,STUDY001-102,102,SITE02,2022-02-15,2022-07-15,2022-02-10,1975-11-07,46,Y,F,Asian,Hispanic,Treatment B,B1,CAN
STUDY001,DM,STUDY001-103,103,SITE01,2022-03-01,2022-08-01,2022-02-25,1990-06-20,31,Y,M,Black,Not Hispanic or Latino,Placebo,P1,USA
STUDY001,DM,STUDY001-104,104,SITE03,2022-01-20,2022-06-20,2022-01-15,1985-01-30,37,Y,F,White,Hispanic,Treatment A,A1,GBR
STUDY001,DM,STUDY001-105,105,SITE02,2022-04-05,2022-09-05,2022-04-01,1965-09-15,56,Y,M,Other,,Placebo,P1,AUS
STUDY001,DM,STUDY001-106,106,SITE03,2022-05-10,2022-10-10,2022-05-05,1995-12-01,26,Y,F,Asian,Not Hispanic or Latino,Treatment B,B1,
STUDY001,DM,STUDY001-107,107,SITE01,2022-06-15,2022-11-15,2022-06-10,1970-03-22,52,Y,F,White,Not Hispanic

### Save as new study folder

In [25]:
import os
import re

def create_new_cdms_study_folder(base_dir="cdms_data"):
    # Make sure base directory exists
    os.makedirs(base_dir, exist_ok=True)
    
    # Find all existing study folders
    existing = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    pattern = re.compile(r"study_(\d+)")
    
    numbers = []
    for d in existing:
        match = pattern.match(d)
        if match:
            numbers.append(int(match.group(1)))
    
    # Determine next number
    if numbers:
        next_number = max(numbers) + 1
    else:
        next_number = 1
    
    # New folder name
    new_folder = f"study_{next_number}"
    new_folder_path = os.path.join(base_dir, new_folder)
    
    # Create the folder
    os.makedirs(new_folder_path, exist_ok=True)
    
    print(f"Created new study folder: {new_folder_path}")
    return new_folder_path


In [26]:
new_study_path = create_new_cdms_study_folder()

Created new study folder: cdms_data/study_1


In [27]:
from io import StringIO

def save_gpt_csvs_to_study_folder(gpt_text_response, study_folder_path):
    # Make sure the target folder exists
    os.makedirs(study_folder_path, exist_ok=True)
    
    domains = ["DM", "VS", "AE", "EX", "TS", "TV"]
    
    for domain in domains:
        start_marker = f"{domain} Dataset:"
        try:
            start_idx = gpt_text_response.index(start_marker) + len(start_marker)
        except ValueError:
            print(f"Warning: {domain} Dataset not found in the GPT response.")
            continue

        # Find next domain start to separate sections
        next_domain_idx = len(gpt_text_response)
        for d in domains:
            if d != domain and f"{d} Dataset:" in gpt_text_response:
                idx = gpt_text_response.index(f"{d} Dataset:")
                if idx > start_idx:
                    next_domain_idx = min(next_domain_idx, idx)

        # Extract CSV text for the domain
        csv_text = gpt_text_response[start_idx:next_domain_idx].strip()

        # Load CSV text into Pandas DataFrame
        try:
            df = pd.read_csv(StringIO(csv_text))
        except Exception as e:
            print(f"Error parsing {domain} CSV: {e}")
            continue

        # Save CSV to the study folder
        file_path = os.path.join(study_folder_path, f"{domain.lower()}.csv")
        df.to_csv(file_path, index=False)
        print(f"Saved {domain} data to {file_path}")

In [28]:
# Save the latest generated dataset into the folder
save_gpt_csvs_to_study_folder(cdms_csv_text, new_study_path)

Saved DM data to cdms_data/study_1/dm.csv
Saved VS data to cdms_data/study_1/vs.csv
Saved AE data to cdms_data/study_1/ae.csv
Saved EX data to cdms_data/study_1/ex.csv
Saved TS data to cdms_data/study_1/ts.csv
Saved TV data to cdms_data/study_1/tv.csv


In [29]:
# For traceability also store the raw model response in a file
def save_raw_gpt_response(gpt_text_response, study_folder_path):
    file_path = os.path.join(study_folder_path, "raw_response.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(gpt_text_response)
    print(f"Saved raw GPT response to {file_path}")

In [30]:
save_raw_gpt_response(cdms_csv_text, new_study_path)

Saved raw GPT response to cdms_data/study_1/raw_response.txt


In [31]:
# For traceability also save prompt
def save_prompt(prompt_text, study_folder_path):
    file_path = os.path.join(study_folder_path, "prompt.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(prompt_text)
    print(f"Saved GPT prompt to {file_path}")

In [33]:
save_prompt(prompt_cdms_dataset, new_study_path)

Saved GPT prompt to cdms_data/study_1/prompt.txt


In [43]:
from datetime import datetime

def save_metadata(study_folder_path, model_name, prompt_version):
    metadata = {
        "timestamp": datetime.now().isoformat(),
        "prompt_version": prompt_version,
        "gpt_model": model_name,
        "number_of_subjects": None
    }
    
    dm_file_path = os.path.join(study_folder_path, "dm.csv")
    if os.path.exists(dm_file_path):
        try:
            dm = pd.read_csv(dm_file_path)
            
            # Only accept SUBJID
            if "SUBJID" in dm.columns:
                metadata["number_of_subjects"] = dm["SUBJID"].nunique()
                print(f"Found {metadata['number_of_subjects']} unique subjects using 'SUBJID'.")
            elif "subjid" in dm.columns:
                metadata["number_of_subjects"] = dm["subjid"].nunique()
                print(f"Found {metadata['number_of_subjects']} unique subjects using 'subjid'.")
            else:
                print(f"No SUBJID field found in DM dataset. Cannot determine number of subjects.")
        
        except Exception as e:
            print(f"Error reading DM dataset: {e}")
    else:
        print(f"DM dataset not found at {dm_file_path}. Skipping subject count.")

    file_path = os.path.join(study_folder_path, "metadata.json")
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=4)
    print(f"Saved metadata to {file_path}")


In [44]:
save_metadata(new_study_path, prompt_version_cdms_dataset, model_name)

Found 12 unique subjects using 'SUBJID'.
Saved metadata to cdms_data/study_1/metadata.json


# 3. Study explorer
Biostatistians need an easy way to browse, select, and explore studies and what sort of SDTM mappings will be needed based on electronic data capture (EDC) source data. 

Before mapping, scientists need to understand the data. They may discover missing domains, missing fields, or formatting issues upfront.

### Show available studies

In [8]:
from IPython.display import display

BASE_DIR = "cdms_data"

def list_studies(base_dir=BASE_DIR):
    studies = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    return studies

# List studies
studies = list_studies()
print("Available studies:")
for idx, study in enumerate(studies, 1):
    print(f"{idx}. {study}")

Available studies:
1. study_1


### Select a study

In [10]:
# Pick a study Id to explore
selected_idx = 0

selected_study = studies[selected_idx]
selected_study_path = os.path.join(BASE_DIR, selected_study)

print(f"Selected: {selected_study}")

Selected: study_1


### Load datasets for the selected study

In [11]:
def load_study_datasets(study_path):
    datasets = {}
    domains = ["dm", "vs", "ae", "ex", "ts", "tv"]
    
    for domain in domains:
        file_path = os.path.join(study_path, f"{domain}.csv")
        if os.path.exists(file_path):
            datasets[domain.upper()] = pd.read_csv(file_path)
        else:
            print(f"{domain}.csv not found in {study_path}")
    
    return datasets

In [29]:
# Load datasets
datasets = load_study_datasets(selected_study_path)

# Collect all fields across all domains
cdms_fields = set()

for domain, df in datasets.items():
    cdms_fields.update(df.columns.tolist())

# Optional: Sort fields for nicer display
cdms_fields = sorted(list(cdms_fields))

print(f"Collected {len(cdms_fields)} fields across study domains.")
print(cdms_fields)

Collected 48 fields across study domains.
['AEACN', 'AEENDTC', 'AEOUT', 'AESEQ', 'AESER', 'AESEV', 'AESTDTC', 'AETERM', 'AGE', 'AGEU', 'ARM', 'ARMCD', 'BRTHDTC', 'COUNTRY', 'DMDTC', 'DOMAIN', 'ETHNIC', 'EXDOSE', 'EXDOSFRQ', 'EXDOSU', 'EXENDTC', 'EXROUTE', 'EXSEQ', 'EXSTDTC', 'EXTRT', 'RACE', 'RFENDTC', 'RFSTDTC', 'SEX', 'SITEID', 'STUDYID', 'SUBJID', 'TSPARM', 'TSPARMCD', 'TSSEQ', 'TSVAL', 'TSVALU', 'TVSTDTC', 'USUBJID', 'VISIT', 'VISITNUM', 'VSBLFL', 'VSDTC', 'VSORRES', 'VSORRESU', 'VSSEQ', 'VSTEST', 'VSTESTCD']


### Show basic dataset info

In [12]:
def show_study_summary(datasets):
    print("\nStudy Summary:")

    for domain, df in datasets.items():
        print(f"\nDomain: {domain}")
        print(f"- Records: {len(df)}")
        print(f"- Columns: {list(df.columns)}")
        print("- Sample records:")
        display(df.head(3))

# Show the summary
show_study_summary(datasets)


Study Summary:

Domain: DM
- Records: 12
- Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'SUBJID', 'SITEID', 'RFSTDTC', 'RFENDTC', 'DMDTC', 'BRTHDTC', 'AGE', 'AGEU', 'SEX', 'RACE', 'ETHNIC', 'ARM', 'ARMCD', 'COUNTRY']
- Sample records:


,STUDYID,DOMAIN,USUBJID,SUBJID,SITEID,RFSTDTC,RFENDTC,DMDTC,BRTHDTC,AGE,AGEU,SEX,RACE,ETHNIC,ARM,ARMCD,COUNTRY
0,STUDY001,DM,STUDY001-101,101,SITE01,2022-01-10,2022-06-10,2022-01-05,1980-04-12,41,Y,M,White,Not Hispanic or Latino,Treatment A,A1,USA
1,STUDY001,DM,STUDY001-102,102,SITE02,2022-02-15,2022-07-15,2022-02-10,1975-11-07,46,Y,F,Asian,Hispanic,Treatment B,B1,CAN
2,STUDY001,DM,STUDY001-103,103,SITE01,2022-03-01,2022-08-01,2022-02-25,1990-06-20,31,Y,M,Black,Not Hispanic or Latino,Placebo,P1,USA



Domain: VS
- Records: 20
- Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'VSSEQ', 'VISITNUM', 'VISIT', 'VSTESTCD', 'VSTEST', 'VSORRES', 'VSORRESU', 'VSBLFL', 'VSDTC']
- Sample records:


,STUDYID,DOMAIN,USUBJID,VSSEQ,VISITNUM,VISIT,VSTESTCD,VSTEST,VSORRES,VSORRESU,VSBLFL,VSDTC
0,STUDY001,VS,STUDY001-101,1,1,Visit 1,WT,Weight,70.2,kg,Y,2022-01-10
1,STUDY001,VS,STUDY001-101,2,2,Visit 2,WT,Weight,70.5,kg,NaN,2022-02-07
2,STUDY001,VS,STUDY001-102,3,1,Visit 1,HT,Height,165.0,cm,Y,2022-02-15



Domain: AE
- Records: 12
- Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'AESEQ', 'AETERM', 'AESTDTC', 'AEENDTC', 'AESEV', 'AESER', 'AEACN', 'AEOUT']
- Sample records:


...

,STUDYID,DOMAIN,USUBJID,EXSEQ,EXTRT,EXDOSE,EXDOSU,EXDOSFRQ,EXROUTE,EXSTDTC,EXENDTC
0,STUDY001,EX,STUDY001-101,1,Treatment A,200,mg,QD,Oral,2022-01-11,2022-01-11
1,STUDY001,EX,STUDY001-101,2,Treatment A,200,mg,QD,Oral,2022-02-11,2022-02-11
2,STUDY001,EX,STUDY001-101,3,Treatment A,200,mg,QD,Oral,2022-03-11,2022-03-11



Domain: TS
- Records: 12
- Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'TSSEQ', 'TSPARMCD', 'TSPARM', 'TSVAL', 'TSVALU']
- Sample records:


,STUDYID,DOMAIN,USUBJID,TSSEQ,TSPARMCD,TSPARM,TSVAL,TSVALU
0,STUDY001,TS,STUDY001-101,1,TRTDAYS,Treatment Duration,59.0,Days
1,STUDY001,TS,STUDY001-102,2,TRTDAYS,Treatment Duration,60.0,Days
2,STUDY001,TS,STUDY001-103,3,TRTDAYS,Treatment Duration,61.0,Days



Domain: TV
- Records: 20
- Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'VISITNUM', 'VISIT', 'TVSTDTC']
- Sample records:


,STUDYID,DOMAIN,USUBJID,VISITNUM,VISIT,TVSTDTC
0,STUDY001,TV,STUDY001-101,1,Screening,2022-01-05
1,STUDY001,TV,STUDY001-101,2,Baseline,2022-01-10
2,STUDY001,TV,STUDY001-102,1,Screening,2022-02-10


### Inspect data with GPT

In [13]:
# Function to create a summary string of the dataset
def generate_study_description(datasets):
    desc = ""
    for domain, df in datasets.items():
        desc += f"\nDomain: {domain}\n"
        desc += f"Fields: {list(df.columns)}\n"
        desc += f"Sample (first 3 rows):\n{df.head(3).to_string(index=False)}\n"
    return desc

In [14]:
# Generate long string describing the study datasets
study_description = generate_study_description(datasets)

In [25]:
# Create prompt for gpt to review study long string
prompt_study_review = f"""
You are a clinical data quality assistant.

You are given CDMS datasets from a clinical study, with the end goal of creating SDTMIG-conformant mapping and analysis dataset for submission to the FDA.  
Your task is to review the quality of source data for later conversion to SDTMIG-compliant dataset. Please perform the following:

1. Summarize the study datasets briefly (domains, number of fields, sample data quality).
2. Identify potential data issues (e.g., wrong formatting, missing values, unexpected field names).
3. Suggest corrections or improvements (e.g., standardize visit codes, fix missing dates).

Datasets:

{study_description}
"""

In [26]:
def study_review(prompt_study_review):
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt_study_review}],
            #temperature=0.2,   # not an option with latest reasoning models
            #max_tokens=1500    # not anoption with latest reasoning models
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error reviewing study: {e}"

In [28]:
# Generate study review for our study
gpt_review = study_review(prompt_study_review)
print(gpt_review)

1. Study Dataset Summary  
• Domains and field counts:  
  – DM (Demographics): 16 fields  
  – VS (Vital Signs): 12 fields  
  – AE (Adverse Events): 10 fields  
  – EX (Exposure): 11 fields  
  – TS (Trial Summary – derived): 8 fields  
  – TV (Trial Visits): 6 fields  

• Overall sample quality:  
  – Dates are in ISO “YYYY-MM-DD” format throughout.  
  – Subjects are consistently identified by USUBJID.  
  – Numeric results in VS and TS are given with units.  
  – Controlled terminology is used in many fields (e.g., AESEV, EXDOSFRQ), but some values deviate slightly from CDISC standards.  

2. Potential Data Issues  

Domain DM  
  • AGEU uses “Y” instead of full CDISC term “Years.”  
  • RACE and ETHNIC values are free-text (“White,” “Not Hispanic or Latino,” “Hispanic”) rather than CDISC controlled terms (e.g., WHITE, ASIAN, NOT HISPANIC OR LATINO).  
  • RFSTDTC (First treatment) post-dates DMDTC (Demography date) in row 1; confirm DMDTC = screening date.  

Domain VS  
  • VSBL

## 4. SDTM mapping agent
Dynamic automapper agent creating a draft mapping specification with trace/chain of thought record. 
In this case, we configure the agent for SDTMIG 3.3.

In [32]:
def generate_field_mappings(fields, model_name="gpt-4o-mini"):
    prompt = f"""
You are an expert SDTMIG 3.3 mapping assistant.

You will be given a list of CDMS source variables. For each source variable:
- Identify the target SDTMIG 3.3 standard variable it should map to.
- Indicate if a transformation is needed (e.g., format change, controlled terminology mapping, case normalization).
- Provide a detailed Chain of Thought (CoT) explanation for why the mapping was made and any transformations suggested.

Respond in a structured JSONL format: one line per field, with the following keys:
- source_field
- target_field
- transformation_needed (true/false)
- transformation_description
- draft_reasoning

Here is the list of fields:
{fields}

Start when ready.
"""
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
        )
    except Exception as e:
        return f"Error reviewing study: {e}"
    
    result = response.choices[0].message.content.strip()
    return result


In [33]:
# Generate field mappings
mapping_output_text = generate_field_mappings(cdms_fields)

# Print for quick review
print(mapping_output_text)

```jsonl
{"source_field":"AEACN","target_field":"AEACN","transformation_needed":false,"transformation_description":"","draft_reasoning":"AEACN directly maps to the corresponding SDTM variable without any transformation."}
{"source_field":"AEENDTC","target_field":"AEENDTC","transformation_needed":false,"transformation_description":"","draft_reasoning":"AEENDTC is already formatted to match SDTM requirements and does not require any changes."}
{"source_field":"AEOUT","target_field":"AEOUT","transformation_needed":false,"transformation_description":"","draft_reasoning":"AEOUT aligns directly with SDTM variables; no transformation is necessary."}
{"source_field":"AESEQ","target_field":"AESEQ","transformation_needed":false,"transformation_description":"","draft_reasoning":"AESEQ is kept as is as it directly corresponds to SDTM."}
{"source_field":"AESER","target_field":"AESER","transformation_needed":false,"transformation_description":"","draft_reasoning":"AESER does not require change; it i

Save mapping trace directly into the selected study folder (e.g., cdms_data/study_1/mapping_draft.jsonl)

In [34]:
def save_mapping_jsonl(mapping_text, study_folder):
    output_path = os.path.join(study_folder, "mapping_draft.jsonl")
    lines = mapping_text.split("\n")
    with open(output_path, "w", encoding="utf-8") as f:
        for line in lines:
            f.write(line.strip() + "\n")
    print(f"Mapping trace saved to {output_path}")

In [35]:
save_mapping_jsonl(mapping_output_text, selected_study_path)

Mapping trace saved to cdms_data/study_1/mapping_draft.jsonl


### Interim results

**Steps performed**
For a given study datasets, we:
1. load supported domains (DM, VS, AE, EX, TS, TV)
2. explore data	and note observations/irregularities in the data
3. collect CDMS source fields dynamically
4. generate initial SDTM mappings including source-to-target variable mapping, confirm whether transformation of values is needed, and CoT reasoning
5. save mapping trace as mapping_draft.jsonl inside the selected study folder

**Result**
* A full, machine-readable, human-readable draft mapping
* For every source field
* With Chain of Thought explainability
* Ready for validation

**Benefit**
Biostatisticians can review this draft as opposed to generating themselves from scratch.
They can also update this workflow to include their existing transformation functions. 
That way the AI agent can identify the function needed for mapping, or suggest transformation code.

### Evaluation

**Summary**

Mapping was performed as expected. The agent was able to parse different value fields (i.e., is more flexible and less maintenance effort than hardcoded automation & rule-based system). However, it was also evident that the generated source dataset hallucinated target variables in the source dataset which reduced the number of needed transformations in the mapping process. The mapping summary is as follows:

|Category	                                    |Examples	                                 |Count         |
|-----------------------------------------------|--------------------------------------------|--------------|
|Direct, no transformation	                    |STUDYID, USUBJID, SEX, AGE, SITEID	         |Majority (~90%)|
|Needs controlled terminology mapping	        |EXROUTE, VISIT, ETHNIC, RACE, possibly AGEU |~5–10 fields  |
|Needs date reformatting check (hidden risk)	|AESTDTC, EXSTDTC, BRTHDTC	                 |~5 fields (could be a silent miss)|

**Needs revision**

|Issue                  |Details	                                        |Example                    |
|-----------------------|---------------------------------------------------|---------------------------|
|Overconfident          |No-Transformation                                  |Some fields like ETHNIC, RACE, and AGEU are marked as no-transformation, but in real SDTMIG, they often need controlled terminology mapping."White" ➔ "WHITE", "Not Hispanic" ➔ "NOT HISPANIC OR LATINO", etc.|
|Generic reasonings	    |Some CoT draft reasonings are a bit shallow (e.g., "FIELD maps directly with no changes"), without checking value distributions.	                         |Would want the validator agent to expand these.|
|Formatting assumptions |GPT assumed that all date fields (e.g., AESTDTC, EXSTDTC, BRTHDTC) are properly formatted without transformation needed. In practice might not be true.     |E.g., even date fields may need reformatting to ISO 8601.|

## 5. SDTM mapping validator
The results from the mapping so far showed a few flaws that need revision step. 
Instead of letting the biostatistician operate on the draft, let's create a validator which automatically verifies and updates the mappings. As with the previous steps, lineage/traceability of record changes is critical and should not be broken. 

In [43]:
# Load mapping draft stored locally in order to then parse draft mappings
def load_mapping_draft(study_folder_path):
    mapping_file = os.path.join(study_folder_path, "mapping_draft.jsonl")
    mappings = []
    with open(mapping_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # Skip empty lines
                try:
                    mappings.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"JSON decode error: {e} on line: {line}")
    return mappings

In [52]:
#Define mapping validator agent
def validate_mapping_entry(mapping_entry, model_name="gpt-4o"):
    prompt = f"""
You are a senior SDTMIG 3.3 mapping validator.

You will review a proposed mapping between a CDMS source field and an SDTM target field.

Here is the draft mapping:
- Source Field: {mapping_entry['source_field']}
- Target Field: {mapping_entry['target_field']}
- Transformation Needed: {mapping_entry['transformation_needed']}
- Transformation Description: {mapping_entry['transformation_description']}
- Draft Reasoning: {mapping_entry['draft_reasoning']}

Your task:
- Confirm whether the source_field should map to the target_field (if not, propose a correction).
- Confirm whether a transformation is needed (if not accurate, correct it).
- Expand or improve the Chain of Thought (CoT) reasoning.
- Be stricter: require controlled terminology mapping if necessary (e.g., RACE, ETHNIC, AGEU).
- Output a structured JSON object with:
  - source_field
  - validated_target_field
  - transformation_needed (true/false)
  - improved_transformation_description
  - improved_reasoning
  - validation_comments

Respond ONLY with valid JSON, no triple backticks nor markdown formatting.
"""
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
        )

        # Check if response is non-empty
        validated_output = response.choices[0].message.content.strip()
        # Simple clean if response has triple backticks
        validated_output = validated_output.replace("```json", "").replace("```", "").strip()
        if not validated_output:
            print(f"Empty response for field {mapping_entry['source_field']}")
            return None

        # Parse JSON safely
        validated_json = json.loads(validated_output)
        return validated_json

    except openai.OpenAIError as e:
        print(f"OpenAI API error for field {mapping_entry['source_field']}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error for field {mapping_entry['source_field']}: {e}")
        print(f"Response content: {validated_output}")
        return None
    except Exception as e:
        print(f"Unexpected error for field {mapping_entry['source_field']}: {e}")
        return None

In [53]:
# Validate all mappings
def validate_all_mappings(mappings, study_folder_path, model_name="gpt-4o"):
    validated_mappings = []
    
    for idx, mapping_entry in enumerate(mappings):
        print(f"Validating field {idx+1}/{len(mappings)}: {mapping_entry['source_field']}...")
        validated_mapping = validate_mapping_entry(mapping_entry, model_name)
        
        if validated_mapping:
            validated_mappings.append(validated_mapping)
        else:
            print(f"Skipping {mapping_entry['source_field']} due to validation failure.")
    
    # Save validated mappings as `mapping_validated.jsonl` file
    output_file = os.path.join(study_folder_path, "mapping_validated.jsonl")
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in validated_mappings:
            f.write(json.dumps(entry) + "\n")
    
    print(f"Validated mappings saved to {output_file}")

In [54]:
# Load draft mappings
draft_mappings = load_mapping_draft(selected_study_path)

# Validate them all
validate_all_mappings(draft_mappings, selected_study_path)

JSON decode error: Expecting value: line 1 column 1 (char 0) on line: ```jsonl
JSON decode error: Expecting value: line 1 column 1 (char 0) on line: ```
Validating field 1/48: AEACN...
Validating field 2/48: AEENDTC...
Validating field 3/48: AEOUT...
Validating field 4/48: AESEQ...
Validating field 5/48: AESER...
Validating field 6/48: AESEV...
Validating field 7/48: AESTDTC...
Validating field 8/48: AETERM...
Validating field 9/48: AGE...
Validating field 10/48: AGEU...
Validating field 11/48: ARM...
Validating field 12/48: ARMCD...
Validating field 13/48: BRTHDTC...
Validating field 14/48: COUNTRY...
Validating field 15/48: DMDTC...
Validating field 16/48: DOMAIN...
Validating field 17/48: ETHNIC...
Validating field 18/48: EXDOSE...
Validating field 19/48: EXDOSFRQ...
Validating field 20/48: EXDOSU...
Validating field 21/48: EXENDTC...
Validating field 22/48: EXROUTE...
Validating field 23/48: EXSEQ...
Validating field 24/48: EXSTDTC...
Validating field 25/48: EXTRT...
Validating fie

### Interim results

Draft mapping was already compliant for structural fields and dates (e.g., names, ISO8601) where no controlled terminology is needed.

**Results**
* The values inside fields need controlled terminology mapping (e.g., free-text → CDISC code).
    Interpretation: Validator is stricter on value-level compliance, not just field names.
* Field names were wrong in draft. Validator correctly rewired the mapping to match SDTMIG.
    Interpretation: Validator catches not only terminology issues but structural mismatches.

**Summary**

|Category	                            |Count	    |Example Fields                       |
|---------------------------------------|-----------|-------------------------------------|
|No transformation needed (confirmed)	|~60%	    |AEENDTC, AESTDTC, AESEQ, BRTHDTC, DMDTC, SITEID, SUBJID, VSSEQ, STUDYID, RFSTDTC      |
|Controlled terminology required	    |~35%	    |RACE, ETHNIC, AEACN, AEOUT, AESEV, AESER, AGEU, EXROUTE, SEX, VISIT, EXDOSFRQ, EXTRT |
|Field remapping required               |~5%	    |TSPARM → TSPARMCD, VSTEST → VSTESTCD |

### Evaluation

The mapping validator agent successfully demonstrates:
* AI model understanding of SDTM rules with sufficient precision e.g., when controlled terminology must be applied
* Detailed CoT reasoning per field similar to human-like reasoning
* SDTMIG 3.3-compliant guidance
* Audit-ready traceability

**Areas of further improvement**
* Practical transformation guidance - ground recommendations in specific set of *predefined transformation functions* (direct_move, lookup from codelist for values not compliant with CDISC, assign, ISO8601)
* Make output more structured to simplify integration with follow-on traditional automation steps, especially by introducing *issue categorization*, listing *correct and incorrect values*, and introducing *codelists/picklists* for internal coding (e.g., CAN=CANADA; 1=Mild,2=Moderate,3=Severe, etc.)

Next, let's enhance our validator and compare output.

In [59]:
# Define SDTM codelists as Python dictionary. This should include controlled terminology and can be supplied from an external system if such metadata is already stored. 
codelists = {
    "AEACN": ["DOSE REDUCED", "DRUG INTERRUPTED", "DRUG WITHDRAWN", "NOT APPLICABLE", "UNKNOWN"],
    "AEOUT": ["FATAL", "NOT RECOVERED/NOT RESOLVED", "RECOVERED/RESOLVED", "RECOVERED/RESOLVED WITH SEQUELAE", "RECOVERING/RESOLVING", "UNKNOWN"],
    "AESEV": ["MILD", "MODERATE", "SEVERE"],
    "AESER": ["Y", "N"],
    "ETHNIC": ["HISPANIC OR LATINO", "NOT HISPANIC OR LATINO", "UNKNOWN", "NOT REPORTED"],
    "RACE": ["AMERICAN INDIAN OR ALASKA NATIVE", "ASIAN", "BLACK OR AFRICAN AMERICAN", "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER", "WHITE", "OTHER", "MULTIPLE"],
    "COUNTRY": ["USA", "GBR", "DEU", "FRA", "ITA", "ESP"],  # Example: ISO-3166-1 alpha-3 country codes
    "EXROUTE": ["ORAL", "INTRAVENOUS", "SUBCUTANEOUS", "TOPICAL", "INHALATION"],
    "SEX": ["M", "F", "U"],
}

In [66]:
# Create enhanced validator
def validate_mapping_entry_enhanced(mapping_entry, model_name="gpt-4o"):
    # Get allowed values if the target field is in codelists
    allowed_values = codelists.get(mapping_entry['target_field'], [])

    prompt = f"""
You are an SDTM 3.3 clinical data validator agent.

You are reviewing a draft mapping from a clinical CDMS source field to an SDTM target field.

Here is the draft mapping:
- Source Field: {mapping_entry['source_field']}
- Draft Target Field: {mapping_entry['target_field']}
- Draft Transformation Needed: {mapping_entry['transformation_needed']}
- Draft Reasoning: {mapping_entry['draft_reasoning']}

Allowed controlled terminology for the target field {mapping_entry['target_field']}:
{allowed_values}

Your job is to validate it and produce a highly structured output with the following fields:

- source_field (str)
- validated_target_field (str)
- transformation_needed (bool)
- transformation_function (one of ["direct_move", "lookup_from_codelist", "assign_constant", "format_to_ISO8601", "derive_sequence"])
- issue_category (one of ["Controlled Terminology Mapping", "Date Format Issue", "Structural Field Mapping", "Value Transformation", "No Issue"])
- incorrect_values (list of examples if applicable)
- correct_values (list of examples if applicable)
- allowed_values_codelist (list, if applicable, else empty)
- improved_transformation_description (str)
- improved_reasoning (str)
- validation_comments (str)

Rules:
- Suggest a transformation function if needed.
- If no transformation is needed, use "direct_move" and "No Issue".
- If the target field uses controlled terminology, validate values against the allowed list.
- Otherwise, leave allowed_values_codelist empty.

IMPORTANT: Respond ONLY with a single pure JSON object. Do NOT wrap it inside triple backticks or markdown formatting.

STRICT COMPLIANCE RULES: 
- Always validate mappings strictly according to the SDTMIG 3.3 standard.
- Always validate fields against SDTMIG 3.3 standards. Do NOT invent new mappings. 
- For COUNTRY, use full country names only. For ETHNIC, reject any non-ethnicity terms.
- Do not assume mappings based on ISO codes, clinical standards, or other common sense unless explicitly required by SDTMIG 3.3.
- All mappings must align strictly with the provided SDTM controlled terminology.
- If you are unsure whether a controlled terminology is specified in SDTMIG 3.3 for a field, assume NO transformation is needed.
- Your job is to enforce SDTMIG 3.3 compliance, not general best practices.
"""

    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
        )
        validated_output = response.choices[0].message.content.strip()

        validated_output = validated_output.replace("```json", "").replace("```", "").strip()

        if not validated_output:
            print(f"Empty response for field {mapping_entry['source_field']}")
            return None

        validated_json = json.loads(validated_output)
        return validated_json

    except openai.OpenAIError as e:
        print(f"OpenAI API error for field {mapping_entry['source_field']}: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error for field {mapping_entry['source_field']}: {e}")
        print(f"Response content: {validated_output}")
        return None
    except Exception as e:
        print(f"Unexpected error for field {mapping_entry['source_field']}: {e}")
        return None



In [67]:
validated_mappings = []

for idx, mapping_entry in enumerate(draft_mappings):
    print(f"Validating field {idx+1}/{len(draft_mappings)}: {mapping_entry['source_field']}...")
    validated_entry = validate_mapping_entry_enhanced(mapping_entry)
    if validated_entry:
        validated_mappings.append(validated_entry)
    else:
        print(f"Skipped {mapping_entry['source_field']} due to validation error.")

# Save output
enhanced_output_file = os.path.join(selected_study_path, "mapping_validated_enhanced.jsonl")
with open(enhanced_output_file, "w", encoding="utf-8") as f:
    for entry in validated_mappings:
        f.write(json.dumps(entry) + "\n")

print(f"Enhanced validated mappings saved to {enhanced_output_file}")

Validating field 1/48: AEACN...
Validating field 2/48: AEENDTC...
Validating field 3/48: AEOUT...
Validating field 4/48: AESEQ...
Validating field 5/48: AESER...
Validating field 6/48: AESEV...
Validating field 7/48: AESTDTC...
Validating field 8/48: AETERM...
Validating field 9/48: AGE...
Validating field 10/48: AGEU...
Validating field 11/48: ARM...
Validating field 12/48: ARMCD...
Validating field 13/48: BRTHDTC...
Validating field 14/48: COUNTRY...
Validating field 15/48: DMDTC...
Validating field 16/48: DOMAIN...
Validating field 17/48: ETHNIC...
Validating field 18/48: EXDOSE...
Validating field 19/48: EXDOSFRQ...
Validating field 20/48: EXDOSU...
Validating field 21/48: EXENDTC...
Validating field 22/48: EXROUTE...
Validating field 23/48: EXSEQ...
Validating field 24/48: EXSTDTC...
Validating field 25/48: EXTRT...
Validating field 26/48: RACE...
Validating field 27/48: RFENDTC...
Validating field 28/48: RFSTDTC...
Validating field 29/48: SEX...
Validating field 30/48: SITEID...


### Final Results

**Summary**  
We now have:
* Structured schema that machines can act on (build actual ETL pipelines)
* Traceable audit trail that a human reviewer can follow
* Clear agent handoff system (draft ➔ validator ➔ transformation executor)
* Diagnostic report capability (how many fields needed what types of fixes)

**Benefits**

|Aspect                     |Evaluation                                                 |
|---------------------------|-----------------------------------------------------------|
|Traceability	            |Full (source ➔ target ➔ reason ➔ function)                 |
|Explainability             |includes chain of thought for each record/variable         |
|Potential for Autocoding	|Structured output easy to pipe into code generation        |

**Areas of improvement**

There were a few notable gaps (now fixed) in previous iterations/prior to tweaking the prompt/instructions to remain compliant with SDTMIG 3.3. This underscores the importance of explicit ruleset definition and supply with the prompt. Among others, the controlled terminology from SDTMIG should be supplied as ruleset. 

|Field	                    |Issue	                                                    |Suggested Correction               |
|---------------------------|-----------------------------------------------------------|-----------------------------------|
|ETHNIC	                    |Minor mistake: the examples show "ASIAN" and "BLACK" mapping to "HISPANIC OR LATINO" / "NOT HISPANIC OR LATINO" — which is wrong.	| "ASIAN" and "BLACK" belong to RACE, not ETHNIC. ETHNIC values should be like "HISPANIC OR LATINO", "NOT HISPANIC OR LATINO", etc. If "ASIAN" or "BLACK" shows up, it's a field mismatch or source system error. Flag it as invalid for ETHNIC.|
|COUNTRY allowed_values_codelist | The validator enlists a codelist which was not supplied by the user : ["USA", "GBR", "DEU", "FRA", "ITA", "ESP"] — these are ISO country codes. | Enforce compliance with SDTMIG3.3 and ONLY the provided codelists.|


---------------------------------------

### Future directions 
This is the end of our notebook. You may want to continue the experimentation in any of the following directions:
1. define list of allowed transformation functions which the agent should reference in the mapping specification to ensure strict compliance with your codebase (e.g., direct_move, get_usubjid, suppqual)
2. rework the synthetic data generator which currently hallucinates SDTMIG output variables in the CDMS source dataset
3. create a migration agent which allows to migrate a dataset to the next version of SDTMIG (in our example we used SDTMIG 3.3 so you could create a migration agent which automatically detects and remediates non-compliance with SDTMIG 3.4)
4. Compare responses with another model such as Anthropic Claude and foundation models trained on clinical data 
5. Build a transformation engine that take the enhanced mapping specification. Assuming the specification has been reviewed by a human in the loop, the engine performs the data transformations for each variable as specified in the source-to-target mapping by invoking the function referenced in the mapping specification e.g., *direct_move* or *lookup_from_codelist*. 